NLP Practical 3

In [85]:
import numpy as np
import os
from random import shuffle
import re
import sklearn as sk
import matplotlib.pyplot as plt
import random

In [86]:
import urllib.request
import zipfile
import lxml.etree

In [87]:
import tensorflow as tf

In [88]:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
    
raw_text = doc.xpath('//content/text()')
raw_label = doc.xpath('//head/keywords/text()')

del doc

In [89]:
talk_sentences = []
talknum = len(raw_text)

for i in range(talknum):
    temp = re.sub(r'\([^)]*\)', '', raw_text[i])
    temp = re.sub(r'\n', '', raw_text[i])
    temp = temp.split('.')
    talk_sentences.append(temp)

In [90]:
print(talk_sentences[0][1])

To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation


In [91]:
talk_sentence_word = []

for talk in talk_sentences:
    temp = []
    for sent in talk:
        tokens = re.sub(r"[^a-z0-9]+", " ", sent.lower()).split()
        temp.append(tokens)#
    talk_sentence_word.append(temp)

In [92]:
talk_sentence_word[0][1]

['to',
 'me',
 'the',
 'real',
 'real',
 'solution',
 'to',
 'quality',
 'growth',
 'is',
 'figuring',
 'out',
 'the',
 'balance',
 'between',
 'two',
 'activities',
 'exploration',
 'and',
 'exploitation']

In [93]:
freq = {}

for talk in talk_sentence_word:
    for sent in talk:
        for word in sent:
            if word in freq:
                freq[word] += 1
            else:
                freq[word] = 1

In [94]:
from collections import Counter

counts_ted_top1000 = []
words_top_ted = []

MC = Counter(freq).most_common(1000)

for word, count in MC:
    counts_ted_top1000.append(count)
    words_top_ted.append(word)

In [95]:
talk_sentence_word_nostop = talk_sentence_word

for stop in words_top_ted[:200]:
    for talk in talk_sentence_word_nostop:
        for sent in talk:
            for word in sent:
                if word == stop:
                    sent.remove(word)

In [96]:
talk_sentence_word_nostop[0][1]

['real',
 'real',
 'solution',
 'quality',
 'growth',
 'figuring',
 'balance',
 'between',
 'activities',
 'exploration',
 'exploitation']

In [97]:
np.shape(talk_sentence_word_nostop)

(2085,)

In [98]:
input_label = []

for i in range(len(raw_label)):
    temp = re.sub(r' ', '', raw_label[i])
    input_label.append(temp.split(','))
    
#print(input_label_prepro)

In [99]:
labels_binary = []

for i in range(len(input_label)):
    temp = np.zeros(3)
    if 'technology' in '~'.join(input_label[i]):
        temp[0] = 1
    if 'entertainment' in '~'.join(input_label[i]):
        temp[1] = 1
    if 'design' in '~'.join(input_label[i]):
        temp[2] = 1
    labels_binary.append(temp)
        
print(input_label[9])
print(labels_binary[:10])

['talks', 'Senses', 'augmentedreality', 'brain', 'computers', 'creativity', 'cyborg', 'demo', 'design', 'engineering', 'entrepreneur', 'innovation', 'interfacedesign', 'invention', 'neuroscience', 'potential', 'prediction', 'productdesign', 'technology', 'visualizations']
[array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  1.]), array([ 1.,  0.,  1.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 0.,  0.,  0.]), array([ 1.,  0.,  1.])]


In [100]:
temp = list(zip(talk_sentence_word_nostop, labels_binary))
random.shuffle(temp)
talk_sentence_word_nostop_shuffle, labels_binary_shuffle = zip(*temp)

In [101]:
labels_onehot = []
for i in range(len(input_label)):
    temp = np.zeros(8)
    temp[labels_binary[i][0]*4+labels_binary[i][1]*2+labels_binary[i][2]*1] = 1
    labels_onehot.append(temp)

/home/scratch/ms16lg2/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [102]:
labels_onehot[:10]

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])]

In [103]:
labels_onehot_shuffle = []
for i in range(len(input_label)):
    temp = np.zeros(8)
    temp[labels_binary_shuffle[i][0]*4+labels_binary_shuffle[i][1]*2+labels_binary_shuffle[i][2]*1] = 1
    labels_onehot_shuffle.append(temp)

/home/scratch/ms16lg2/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [104]:
train_data = talk_sentence_word_nostop[:1835]
test_data = talk_sentence_word_nostop[1835:]

train_labels = labels_binary[:1835]
test_labels = labels_binary[1835:]

train_labels_onehot = labels_onehot[:1835]
test_labels_onehot = labels_onehot[1835:]

In [105]:
train_data_shuffle = talk_sentence_word_nostop_shuffle[:1835]
test_data_shuffle = talk_sentence_word_nostop_shuffle[1835:]

train_labels_shuffle = labels_binary_shuffle[:1835]
test_labels_shuffle = labels_binary_shuffle[1835:]

train_labels_onehot_shuffle = labels_onehot_shuffle[:1835]
test_labels_onehot_shuffle = labels_onehot_shuffle[1835:]

In [106]:
print(np.shape(talk_sentence_word_nostop))
print(np.shape(labels_binary))
print(np.shape(labels_onehot))
print(np.shape(train_data))
print(np.shape(train_labels))
print(np.shape(train_labels_onehot))
print(np.shape(test_data))
print(np.shape(test_labels))
print(np.shape(test_labels_onehot))

(2085,)
(2085, 3)
(2085, 8)
(1835,)
(1835, 3)
(1835, 8)
(250,)
(250, 3)
(250, 8)


In [107]:
print(np.shape(talk_sentence_word_nostop_shuffle))
print(np.shape(labels_binary_shuffle))
print(np.shape(labels_onehot_shuffle))
print(np.shape(train_data_shuffle))
print(np.shape(train_labels_shuffle))
print(np.shape(train_labels_onehot_shuffle))
print(np.shape(test_data_shuffle))
print(np.shape(test_labels_shuffle))
print(np.shape(test_labels_onehot_shuffle))

(2085,)
(2085, 3)
(2085, 8)
(1835,)
(1835, 3)
(1835, 8)
(250,)
(250, 3)
(250, 8)


In [108]:
check = np.asarray(train_labels_onehot)
print(check.sum(0) / check.sum())
check = np.asarray(test_labels_onehot)
print(check.sum(0) / check.sum())

[ 0.57493188  0.0746594   0.0719346   0.00980926  0.17711172  0.06376022
  0.01525886  0.01253406]
[ 0.292  0.108  0.164  0.024  0.224  0.112  0.032  0.044]


In [109]:
check = np.asarray(train_labels_onehot_shuffle)
print(check.sum(0) / check.sum())
check = np.asarray(test_labels_onehot_shuffle)
print(check.sum(0) / check.sum())

[ 0.53787466  0.07901907  0.08174387  0.0119891   0.18692098  0.06594005
  0.01907357  0.01743869]
[ 0.564  0.076  0.092  0.008  0.152  0.096  0.004  0.008]


In [110]:
train_lenghts = []

for talk in train_data_shuffle:
    N = 0
    for sent in talk:
        for word in sent:
            N += 1
    train_lenghts.append(N)

In [111]:
train_lenghts[:10]

[973, 429, 1002, 764, 758, 615, 842, 1195, 1071, 1003]

In [112]:
test_lenghts = []

for talk in test_data_shuffle:
    N = 0
    for sent in talk:
        for word in sent:
            N += 1
    test_lenghts.append(N)

In [113]:
test_lenghts[:10]

[785, 552, 309, 915, 861, 420, 771, 503, 941, 769]

In [114]:
train_talk_word = []

for talk in train_data_shuffle:
    temp = []
    for sent in talk:
        for word in sent:
            temp.append(word)
    train_talk_word.append(temp)
    
print(len(train_talk_word[0]))

973


In [115]:
test_talk_word = []

for talk in test_data_shuffle:
    temp = []
    for sent in talk:
        for word in sent:
            temp.append(word)
    test_talk_word.append(temp)
    
print(len(test_talk_word[0]))

785


In [116]:
test_talk_word[0][:10]

['portuguese',
 'arrived',
 'latin',
 'america',
 '500',
 'obviously',
 'found',
 'amazing',
 'tropical',
 'forest']

In [117]:
train_data[0][:5]

[['reasons', 'companies', 'fail'],
 ['real',
  'real',
  'solution',
  'quality',
  'growth',
  'figuring',
  'balance',
  'between',
  'activities',
  'exploration',
  'exploitation'],
 ['both', 'necessary', 'too'],
 ['consider', 'facit'],
 ['old', 'enough', 'remember']]

In [118]:
vocab = {}
N = 1

for talk in train_data_shuffle:
    for sent in talk:
        for word in sent:
            if word in vocab:
                N = N
            else:
                vocab[word] = N
                N += 1
                
vocab["unknown_word"] = 0

In [119]:
vocab["unknown_word"]

0

In [120]:
train_talk_word_index = []

for talk in train_talk_word:
    temp = []
    for word in talk:
        if word in vocab:
            temp.append(vocab[word])
        else:
            temp.append(vocab["unknown_word"])
    train_talk_word_index.append(temp)

In [121]:
test_talk_word_index = []

for talk in test_talk_word:
    temp = []
    for word in talk:
        if word in vocab:
            temp.append(vocab[word])
        else:
            temp.append(vocab["unknown_word"])
    test_talk_word_index.append(temp)

In [122]:
train_talk_word_index[0][:20]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 7, 3, 11, 12, 3, 13, 14, 13, 15, 13]

In [123]:
train_talk_word_index_exp = []

for i in range(len(train_talk_word_index)):
    temp = []
    for j in range(max(train_lenghts)):
        if j <= (train_lenghts[i]-1):
            temp.append(train_talk_word_index[i][j])
        else:
            temp.append(0)
    train_talk_word_index_exp.append(temp)

In [124]:
test_talk_word_index_exp = []

for i in range(len(test_talk_word_index)):
    temp = []
    for j in range(max(train_lenghts)):
        if j <= (test_lenghts[i]-1):
            temp.append(test_talk_word_index[i][j])
        else:
            temp.append(0)
    test_talk_word_index_exp.append(temp)

In [125]:
np.shape(test_talk_word_index_exp)

(250, 2221)

In [126]:
np.shape(train_talk_word_index_exp)

(1835, 2221)

In [127]:
test_talk_word_index_exp[0]

[17771,
 44,
 6288,
 1426,
 524,
 1668,
 967,
 1976,
 12727,
 4004,
 281,
 7997,
 132,
 967,
 6169,
 727,
 505,
 1701,
 6169,
 1612,
 21004,
 936,
 644,
 17309,
 2023,
 21754,
 5360,
 7164,
 6455,
 6169,
 0,
 0,
 254,
 834,
 634,
 834,
 0,
 1279,
 12097,
 18,
 192,
 5540,
 1855,
 2636,
 31582,
 12097,
 281,
 2625,
 4004,
 335,
 3262,
 335,
 4004,
 2162,
 4841,
 12097,
 106,
 3635,
 15161,
 227,
 7211,
 2360,
 4004,
 6742,
 15907,
 7006,
 180,
 435,
 439,
 307,
 6742,
 2158,
 1424,
 2158,
 3021,
 772,
 433,
 1470,
 435,
 439,
 10333,
 307,
 2044,
 0,
 65,
 588,
 1219,
 2163,
 1570,
 3152,
 1476,
 2295,
 39,
 435,
 18668,
 9251,
 179,
 288,
 2350,
 3251,
 12097,
 1024,
 1839,
 15695,
 10008,
 6577,
 5749,
 4132,
 5749,
 435,
 18668,
 1602,
 7997,
 613,
 274,
 1011,
 12743,
 437,
 1303,
 65,
 435,
 18668,
 4004,
 274,
 878,
 746,
 435,
 18668,
 878,
 748,
 746,
 502,
 16391,
 17146,
 10008,
 9251,
 17146,
 12727,
 9251,
 748,
 746,
 543,
 4004,
 4132,
 746,
 878,
 10008,
 9251,
 1168,
 20

In [128]:
train_labels_RNN = np.argmax(train_labels_onehot_shuffle, 1) 

In [129]:
train_labels_RNN[:10]

array([4, 4, 6, 0, 0, 3, 0, 4, 0, 0])

In [130]:
test_labels_RNN = np.argmax(test_labels_onehot_shuffle, 1) 

In [131]:
class SimpleDataIterator():
    def __init__(self, texts, lengths, labels):
        self.texts = texts
        self.lengths = lengths
        self.labels = labels
        self.size = len(self.texts)
        self.epochs = 0
        self.shuffle()

    def shuffle(self):
        temp = list(zip(self.texts, self.lengths, self.labels))
        random.shuffle(temp)
        self.texts, self.lengths, self.labels = zip(*temp)
        self.cursor = 0

    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epochs += 1
            self.shuffle()
        res_texts = self.texts[self.cursor:self.cursor+n]
        res_lengths = self.lengths[self.cursor:self.cursor+n]
        res_labels = self.labels[self.cursor:self.cursor+n]
        self.cursor += n
        return res_texts, res_labels, res_lengths

In [132]:
data = SimpleDataIterator(train_talk_word_index_exp, train_lenghts, train_labels_RNN)
d = data.next_batch(3)
print('Input sequences\n', d[0], end='\n\n')
print('Target values\n', d[1], end='\n\n')
print('Sequence lengths\n', d[2])

Input sequences
 ([2926, 2809, 1855, 6264, 111, 10735, 24652, 3683, 1462, 2677, 3383, 1825, 1850, 554, 148, 149, 148, 6252, 746, 1855, 22923, 11563, 24653, 309, 826, 8103, 2921, 3365, 3713, 6270, 826, 1153, 2921, 9744, 1009, 24654, 2424, 6270, 826, 24655, 2390, 3434, 1631, 24656, 3914, 10769, 1008, 4878, 1426, 852, 7847, 6270, 924, 3330, 1957, 9745, 6264, 111, 4052, 253, 824, 253, 3903, 2750, 490, 24657, 2368, 10735, 792, 6264, 24657, 10735, 2269, 24658, 1233, 3095, 24659, 16145, 2584, 127, 16145, 10337, 706, 8916, 170, 1345, 5811, 1981, 9567, 18489, 1, 1124, 13274, 2357, 263, 1957, 1012, 10735, 858, 199, 8818, 8817, 158, 13375, 24660, 199, 8817, 22923, 967, 378, 967, 578, 2110, 1957, 1038, 2110, 1668, 16516, 442, 2893, 3048, 7280, 826, 6443, 442, 24657, 356, 730, 9745, 1018, 2921, 3365, 6166, 17500, 5381, 10764, 24661, 3642, 1952, 2548, 3291, 967, 3914, 1419, 753, 318, 2506, 1227, 24662, 333, 2110, 24657, 3989, 24663, 6270, 1748, 6270, 12927, 967, 17500, 1957, 4157, 1345, 4157, 24664,

In [133]:
np.shape(train_talk_word_index_exp)

(1835, 2221)

In [134]:
np.shape(test_talk_word_index_exp)

(250, 2221)

In [135]:
np.shape(train_lenghts)

(1835,)

In [136]:
np.shape(test_lenghts)

(250,)

In [137]:
np.shape(train_labels_RNN)

(1835,)

In [138]:
np.shape(test_labels_RNN)

(250,)

In [139]:
len(vocab)

51962

In [56]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

def build_graph(
    vocab_size = len(vocab)+1,
    state_size = 100,
    batch_size = 50,
    num_classes = 8):

    reset_graph()

    # Placeholders
    x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
    seqlen = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
    keep_prob = tf.constant(1.0)

    # Embedding layer
    embeddings = tf.get_variable('embedding_matrix', [vocab_size, state_size])
    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    # RNN
    cell = tf.nn.rnn_cell.LSTMCell(state_size)
    init_state = tf.get_variable('init_state', [1, 2*state_size],
                                 initializer=tf.constant_initializer(0.0))
    init_state = tf.tile(init_state, [batch_size, 1])
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, sequence_length=seqlen,
                                                  initial_state=init_state, dtype=tf.float32)

    # Add dropout, as the model otherwise quickly overfits
    #rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)
    
    """
    Obtain the last relevant output. The best approach in the future will be to use:

        last_rnn_output = tf.gather_nd(rnn_outputs, tf.pack([tf.range(batch_size), seqlen-1], axis=1))

    which is the Tensorflow equivalent of numpy's rnn_outputs[range(30), seqlen-1, :], but the
    gradient for this op has not been implemented as of this writing.

    The below solution works, but throws a UserWarning re: the gradient.
    """
    #idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (seqlen - 1)
    #last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, state_size]), idx)
    
    last_rnn_output = tf.reduce_mean(rnn_outputs, 1)

    # Softmax layer
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(last_rnn_output, W) + b
    preds = tf.nn.softmax(logits)
    correct = tf.equal(tf.cast(tf.argmax(preds,1),tf.int32), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    return {
        'x': x,
        'seqlen': seqlen,
        'y': y,
        'dropout': keep_prob,
        'loss': loss,
        'ts': train_step,
        'preds': preds,
        'accuracy': accuracy
    }

def train_graph(graph, batch_size = 50, num_epochs = 10, iterator = SimpleDataIterator):
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        tr = iterator(train_talk_word_index_exp, train_lenghts, train_labels_RNN)
        te = iterator(test_talk_word_index_exp, test_lenghts, test_labels_RNN)

        step, accuracy = 0, 0
        tr_losses, te_losses = [], []
        current_epoch = 0
        while current_epoch < num_epochs:
            step += 1
            batch = tr.next_batch(batch_size)
            feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2], g['dropout']: 0.6}
            accuracy_, _ = sess.run([g['accuracy'], g['ts']], feed_dict=feed)
            accuracy += accuracy_

            if tr.epochs > current_epoch:
                current_epoch += 1
                tr_losses.append(accuracy / step)
                step, accuracy = 0, 0

                #eval test set
                te_epoch = te.epochs
                while te.epochs == te_epoch:
                    step += 1
                    batch = te.next_batch(batch_size)
                    feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
                    accuracy_ = sess.run([g['accuracy']], feed_dict=feed)[0]
                    accuracy += accuracy_

                te_losses.append(accuracy / step)
                step, accuracy = 0,0
                print("Accuracy after epoch", current_epoch, " - tr:", tr_losses[-1], "- te:", te_losses[-1])

    return tr_losses, te_losses

In [57]:
g = build_graph(vocab_size=len(vocab)+1, batch_size=50, num_classes=8, state_size=100)

In [58]:
tr_losses, te_losses = train_graph(g, batch_size = 50, num_epochs = 10)

Accuracy after epoch 1  - tr: 0.517837840762 - te: 0.593333333731
Accuracy after epoch 2  - tr: 0.532222221295 - te: 0.576000010967
Accuracy after epoch 3  - tr: 0.537222221494 - te: 0.588000011444
Accuracy after epoch 4  - tr: 0.535555551449 - te: 0.571999996901
Accuracy after epoch 5  - tr: 0.534444442226 - te: 0.596000003815
Accuracy after epoch 6  - tr: 0.536111108959 - te: 0.596000003815
Accuracy after epoch 7  - tr: 0.531111109588 - te: 0.603999996185
Accuracy after epoch 8  - tr: 0.532222222951 - te: 0.579999995232
Accuracy after epoch 9  - tr: 0.539444446564 - te: 0.576000005007
Accuracy after epoch 10  - tr: 0.533333338797 - te: 0.583999991417


In [56]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

def build_graph(
    vocab_size = len(vocab)+1,
    state_size = 100,
    batch_size = 50,
    num_classes = 8):

    reset_graph()

    # Placeholders
    x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
    seqlen = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
    keep_prob = tf.constant(1.0)

    # Embedding layer
    embeddings = tf.get_variable('embedding_matrix', [vocab_size, state_size])
    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    # RNN
    fw_cell = tf.contrib.rnn.LSTMCell(state_size)
    bw_cell = tf.contrib.rnn.LSTMCell(state_size)
    init_state = tf.get_variable('init_state', [1, 2*state_size],
                                 initializer=tf.constant_initializer(0.0))
    init_state = tf.tile(init_state, [batch_size, 1])
    rnn_outputs, final_state = tf.nn.bidirectional_dynamic_rnn(fw_cell, bw_cell, rnn_inputs, sequence_length=seqlen,
                                                      dtype=tf.float32)
    
    # Add dropout, as the model otherwise quickly overfits
    #rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)

    #idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (seqlen - 1)
    #last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, state_size]), idx)
    
    print(np.shape(rnn_outputs[0]))
    
    last_rnn_output = tf.concat([rnn_outputs[0], rnn_outputs[1]], 2)
    
    
    last_rnn_output = tf.reduce_mean(last_rnn_output, 1)
    print(np.shape(last_rnn_output))


    # Softmax layer
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [2*state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(last_rnn_output, W) + b
    preds = tf.nn.softmax(logits)
    correct = tf.equal(tf.cast(tf.argmax(preds,1),tf.int32), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    return {
        'x': x,
        'seqlen': seqlen,
        'y': y,
        'dropout': keep_prob,
        'loss': loss,
        'ts': train_step,
        'preds': preds,
        'accuracy': accuracy
    }
    
def train_graph(graph, batch_size = 50, num_epochs = 10, iterator = SimpleDataIterator):
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        tr = iterator(train_talk_word_index_exp, train_lenghts, train_labels_RNN)
        te = iterator(test_talk_word_index_exp, test_lenghts, test_labels_RNN)

        step, accuracy = 0, 0
        tr_losses, te_losses = [], []
        current_epoch = 0
        while current_epoch < num_epochs:
            step += 1
            batch = tr.next_batch(batch_size)
            feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2], g['dropout']: 0.6}
            accuracy_, _ = sess.run([g['accuracy'], g['ts']], feed_dict=feed)
            accuracy += accuracy_

            if tr.epochs > current_epoch:
                current_epoch += 1
                tr_losses.append(accuracy / step)
                step, accuracy = 0, 0

                #eval test set
                te_epoch = te.epochs
                while te.epochs == te_epoch:
                    step += 1
                    batch = te.next_batch(batch_size)
                    feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
                    accuracy_ = sess.run([g['accuracy']], feed_dict=feed)[0]
                    accuracy += accuracy_

                te_losses.append(accuracy / step)
                step, accuracy = 0,0
                print("Accuracy after epoch", current_epoch, " - tr:", tr_losses[-1], "- te:", te_losses[-1])

    return tr_losses, te_losses

In [57]:
g = build_graph(vocab_size=len(vocab)+1, batch_size=50, num_classes=8, state_size=100)

(50, ?, 100)
(50, 200)


In [58]:
tr_losses, te_losses = train_graph(g, batch_size = 50, num_epochs = 10)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Accuracy after epoch 1  - tr: 0.531351346019 - te: 0.543333331744
Accuracy after epoch 2  - tr: 0.536111111442 - te: 0.520000010729
Accuracy after epoch 3  - tr: 0.543888888425 - te: 0.539999997616
Accuracy after epoch 4  - tr: 0.536111106475 - te: 0.580000007153
Accuracy after epoch 5  - tr: 0.54666666521 - te: 0.512000012398
Accuracy after epoch 6  - tr: 0.538888887399 - te: 0.524000000954
Accuracy after epoch 7  - tr: 0.543888889253 - te: 0.551999992132
Accuracy after epoch 8  - tr: 0.540555556615 - te: 0.55600001812
Accuracy after epoch 9  - tr: 0.543333335055 - te: 0.532000005245
Accuracy after epoch 10  - tr: 0.542777774235 - te: 0.543999999762


In [167]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

def build_graph(
    vocab_size = len(vocab)+1,
    state_size = 100,
    batch_size = 50,
    hidden_states = 30,
    num_classes = 8):

    reset_graph()

    # Placeholders
    x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
    seqlen = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
    keep_prob = tf.constant(1.0)

    # Embedding layer
    embeddings = tf.get_variable('embedding_matrix', [vocab_size, state_size])
    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    # RNN
    fw_cell = tf.contrib.rnn.LSTMCell(state_size)
    bw_cell = tf.contrib.rnn.LSTMCell(state_size)
    init_state = tf.get_variable('init_state', [1, 2*state_size],
                                 initializer=tf.constant_initializer(0.0))
    init_state = tf.tile(init_state, [batch_size, 1])
    rnn_outputs, final_state = tf.nn.bidirectional_dynamic_rnn(fw_cell, bw_cell, rnn_inputs, sequence_length=seqlen,
                                                      dtype=tf.float32)
    
    #print(np.shape(rnn_outputs))
    
    #rnn_outputs = tf.concat([rnn_outputs[0], rnn_outputs[1]], 2)
    
    #print(np.shape(rnn_outputs))
    
    # Add dropout, as the model otherwise quickly overfits
    #rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)
    
    #print(np.shape(rnn_outputs))

    #idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (seqlen - 1)
    #last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, 2*state_size]), idx)
    
    #print(np.shape(last_rnn_output))
    
    #print(np.shape(rnn_outputs[0]))
    
    last_rnn_output = tf.concat([rnn_outputs[0], rnn_outputs[1]], 2)
    
    
    last_rnn_output = tf.reduce_mean(last_rnn_output, 1)
    #print(np.shape(last_rnn_output))

    # Hidden layer
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [2*state_size, hidden_states])
        b = tf.get_variable('b', [hidden_states], initializer=tf.constant_initializer(0.0))
    h = tf.matmul(last_rnn_output, W) + b
    
    # Softmax layer
    with tf.variable_scope('softmax'):
        V = tf.get_variable('V', [hidden_states, num_classes])
        c = tf.get_variable('c', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(h, V) + c
    preds = tf.nn.softmax(logits)
    correct = tf.equal(tf.cast(tf.argmax(preds,1),tf.int32), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    return {
        'x': x,
        'seqlen': seqlen,
        'y': y,
        'dropout': keep_prob,
        'loss': loss,
        'ts': train_step,
        'preds': preds,
        'accuracy': accuracy
    }
    
def train_graph(graph, batch_size = 50, num_epochs = 10, iterator = SimpleDataIterator):
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        tr = iterator(train_talk_word_index_exp, train_lenghts, train_labels_RNN)
        te = iterator(test_talk_word_index_exp, test_lenghts, test_labels_RNN)

        step, accuracy = 0, 0
        tr_losses, te_losses = [], []
        current_epoch = 0
        while current_epoch < num_epochs:
            step += 1
            batch = tr.next_batch(batch_size)
            feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2], g['dropout']: 0.6}
            accuracy_, _ = sess.run([g['accuracy'], g['ts']], feed_dict=feed)
            accuracy += accuracy_

            if tr.epochs > current_epoch:
                current_epoch += 1
                tr_losses.append(accuracy / step)
                step, accuracy = 0, 0

                #eval test set
                te_epoch = te.epochs
                while te.epochs == te_epoch:
                    step += 1
                    batch = te.next_batch(batch_size)
                    feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
                    accuracy_ = sess.run([g['accuracy']], feed_dict=feed)[0]
                    accuracy += accuracy_

                te_losses.append(accuracy / step)
                step, accuracy = 0,0
                print("Accuracy after epoch", current_epoch, " - tr:", tr_losses[-1], "- te:", te_losses[-1])

    return tr_losses, te_losses

In [168]:
g = build_graph(vocab_size=len(vocab)+1, batch_size=50, num_classes=8, state_size=100)

In [169]:
tr_losses, te_losses = train_graph(g, batch_size = 50, num_epochs = 10)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Accuracy after epoch 1  - tr: 0.525945946373 - te: 0.57333334287
Accuracy after epoch 2  - tr: 0.540000002417 - te: 0.540000003576
Accuracy after epoch 3  - tr: 0.531666667925 - te: 0.596000003815
Accuracy after epoch 4  - tr: 0.536111109787 - te: 0.548000007868
Accuracy after epoch 5  - tr: 0.547222222719 - te: 0.543999993801
Accuracy after epoch 6  - tr: 0.533333336314 - te: 0.587999999523
Accuracy after epoch 7  - tr: 0.539444444908 - te: 0.572000002861
Accuracy after epoch 8  - tr: 0.536111110614 - te: 0.539999997616
Accuracy after epoch 9  - tr: 0.537222225633 - te: 0.580000001192
Accuracy after epoch 10  - tr: 0.565555558436 - te: 0.544000005722


RNNs have an advantage because for the words are inputted in a sequence therefore there is a possiblity that the NN can actually infer something from the sequence rather than just the bag of words representation which completelly ignores order and context. However, this method takes significantly longer and is even more suceptable to data availablility, because contex is now something else that needs to learned and it requires extra data.

Using just the final state as the word representation actually started off better than bag of words but ended up just predicting ooo. Dropout seems to be the biggest help.

After trying different architectures (LSTM, GRU, Standard RNN cell) no significant differences were found between the model accuracies. We blame it on the data once more.

Biderectional LSTM also didnt seem to help much and in fact did worse

After adding an additional hidden layer the result stayed the same.